In [1]:
from pathlib import Path
from os.path import join
from google.cloud import bigquery
from google.oauth2 import service_account


path = Path(__file__).resolve()
ROOT_PATH = path.parents[2]
ENV_PATH = ROOT_PATH / 'env'

# Credential are stored away from the project folder
CREDENTIALS_PATH = str(ENV_PATH / 'google_gfw_credentials.json')

# 1) Google BigQuery on fishing efforts (data from Gloabl Fishing Watch)
## 1.1) Instantiate Google BigQuery object that will perform queries

Data from Global Fishing Watch about vessels' fishing efforts have been downloaded from https://globalfishingwatch.org/data-download/datasets/public-fishing-effort. The ``fishing-vessels-v2.xlsx`` csv file has been processed and uploaded to Google BigQuery. Then an API wrapper has been generated to be able to access from outside while using private credentials.

In [2]:
credentials = service_account.Credentials.from_service_account_file( CREDENTIALS_PATH, scopes=["https://www.googleapis.com/auth/cloud-platform"],)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

## 1.2) Find total number of vessel with flags

Of the three fields ``flag_gfw``, ``flag_registry`` and ``flag_ais`` find which one has more boats identified

In [3]:
table = "gfw-testing-377510.fishing_efforts.fishing_data"
query = f"""
SELECT
  CONCAT(FORMAT ("%'.2f", SUM(CASE
          WHEN flag_gfw IS NOT NULL THEN 1
        ELSE
        0
      END
        ) / COUNT(*) * 100), '%') AS num_flag_gfw,
  CONCAT(FORMAT ("%'.2f", SUM(CASE
          WHEN flag_ais IS NOT NULL THEN 1
        ELSE
        0
      END
        ) / COUNT(*) * 100), '%') AS num_flag_ais,
  CONCAT(FORMAT ("%'.2f", SUM(CASE
          WHEN flag_registry IS NOT NULL THEN 1
        ELSE
        0
      END
        ) / COUNT(*) * 100), '%') AS num_flag_registry
FROM
  {table}
  """
query_job = client.query(query)

In [4]:
query_job.to_dataframe()

,num_flag_gfw,num_flag_ais,num_flag_registry
0,99.22%,79.79%,21.59%


## 1.3) Find what countries had the most accumulated fishing hours

Next query obtains the list of countries, fishing gear and total fishing hours per country and gear and country. All gears for all countries are fetched but the results are sorted by the total fishing hours per gear in descending order. We use ``flag_gfw`` as it was the flag that identified the most vessels. 


In [9]:
query = f"""
SELECT
  flag_gfw,
  vessel_class_gfw,
  SUM(fishing_hours_2012 + fishing_hours_2013 + fishing_hours_2014 + fishing_hours_2015 + fishing_hours_2016 + fishing_hours_2017 + fishing_hours_2018 + fishing_hours_2019 + fishing_hours_2020) AS total_fishing_hours
FROM
  {table}
WHERE
  flag_gfw IS NOT NULL
  AND vessel_class_gfw IS NOT NULL
GROUP BY
  flag_gfw,
  vessel_class_gfw
HAVING
  total_fishing_hours IS NOT NULL
ORDER BY
  total_fishing_hours DESC,
  flag_gfw
"""
query_job = client.query(query)

In [11]:
query_job.to_dataframe().head(15)

,flag_gfw,vessel_class_gfw,total_fishing_hours
0,FRA,trawlers,6323317.82
1,RUS,trawlers,5230893.76
2,KOR,drifting_longlines,4225950.01
3,CHN,trawlers,3085546.93
4,ESP,drifting_longlines,3071283.75
5,JPN,drifting_longlines,2822906.47
6,ITA,trawlers,2491535.95
7,ESP,trawlers,2412337.95
8,NLD,trawlers,2299255.37
9,DNK,trawlers,2156840.78


## 1.4) Find the gear per country that had the most accmulated fishing ours over the 2012-2020 period

Contrary to the the earlier query, the next one will find the one gear per country with most accummulated fishing hours over the period 2012-2020. No country is therefore repeated in the list. The resulting list is sorted by the number of fishing hours in descending order. 

To avoid performance issues a **group-wise max query** has been used instead of subqueries.


In [15]:
most_fishing_hours_query = f"""
SELECT
    flag_gfw,
    vessel_class_gfw,
    SUM(fishing_hours_2012 + fishing_hours_2013 + fishing_hours_2014 + fishing_hours_2015 + fishing_hours_2016 + fishing_hours_2017 + fishing_hours_2018 + fishing_hours_2019 + fishing_hours_2020) AS total_fishing_hours
  FROM
    {table}
  WHERE
    flag_gfw IS NOT NULL
    AND vessel_class_gfw IS NOT NULL
  GROUP BY
    flag_gfw,
    vessel_class_gfw
  HAVING
    total_fishing_hours IS NOT NULL
"""

query = f"""
SELECT
  q1.flag_gfw,
  q1.vessel_class_gfw,
  FORMAT("%'.2f", q1.total_fishing_hours) as total_fishing_hours
  -- first query calculating most fishing hours per gear and country
FROM ({most_fishing_hours_query}) q1
LEFT JOIN (
    -- second query calculating most fishing hours per gear and country
  {most_fishing_hours_query}) q2
  -- together with left join, for cases where the 'on' condition does not fully apply, q1 results will still be added, however with is counterpart q2 results set as null ...
ON
  q1.flag_gfw = q2.flag_gfw
  AND q1.total_fishing_hours < q2.total_fishing_hours
WHERE
  q2.flag_gfw IS NULL -- ... and the the records with the q2 null values associated are the largest q1 values
ORDER BY
  q1.total_fishing_hours desc,
  q1.flag_gfw
"""
query_job = client.query(query)

In [16]:
results = query_job.to_dataframe()
results.head(len(results))

,flag_gfw,vessel_class_gfw,total_fishing_hours
0,FRA,trawlers,"6,323,317.82"
1,RUS,trawlers,"5,230,893.76"
2,KOR,drifting_longlines,"4,225,950.01"
3,CHN,trawlers,"3,085,546.93"
4,ESP,drifting_longlines,"3,071,283.75"
5,JPN,drifting_longlines,"2,822,906.47"
6,ITA,trawlers,"2,491,535.95"
7,NLD,trawlers,"2,299,255.37"
8,DNK,trawlers,"2,156,840.78"
9,ARG,trawlers,"1,855,625.76"


# 2) Movement Ecology and Graph Theory

In [2]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


UsageError: Line magic function `%%R` not found.
